In [320]:
"""
!pip install geocoder
!pip install meteostat
!pip install geopandas
!pip install osmnx
"""

'\n!pip install geocoder\n!pip install meteostat\n!pip install geopandas\n!pip install osmnx\n'

In [321]:
import os
import requests
from datetime import datetime, timedelta, date
import pandas as pd
from google.colab import files
import geocoder
from geopy.geocoders import Nominatim
from tqdm import tqdm
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import tkinter as tk
from tkinter import messagebox
import ipywidgets as widgets
from IPython.display import display
from dateutil import parser
import ipywidgets as widgets
from IPython.display import display
from meteostat import Stations, Daily
import ee
import geemap

In [322]:
 N = 14 # we want to see the risk index in the future 7 days
 year = 2022
 loc_num= 10

***Input*** ***1: Crop Information: Names & Locations***

In [323]:
df = pd.read_excel("harvest_locations.xlsx", skiprows=1)

# state_location = df["Kt."]
# origin_location = df["Standort"]
crop_names = df.columns[7:].tolist()

mapping = {
    'Brotweizen': 'Wheat',
    'Umstellmahlweizen': 'Wheat',
    'Futterweizen': 'Wheat',
    'Futtergerste': 'Barley',
    'Körnermais': 'Corn',
    'Sojabohnen Futter': 'Soybean',
    'Sojabohnen "Tofu"': 'Soybean',
    'Raps Kl.': 'Rapeseed',
    'Raps HOLL': 'Rapeseed',
    'Sonnenblumen lino': 'Sunflowerseed',
    'Sonnenblumen HO': 'Sunflowerseed'
}

In [324]:
swiss_crop = [crop for crop in crop_names if crop in mapping]
locations_dict = {}

for crop in swiss_crop:
    category = mapping[crop]
    crop_locations = df.loc[df[crop] == "x", "Standort"].tolist()
    crop_locations = crop_locations[:loc_num]
    if category not in locations_dict:
        locations_dict[category] = crop_locations
    else:
        combined = list(dict.fromkeys(locations_dict[category] + crop_locations))
        locations_dict[category] = combined[:loc_num]
for k, v in locations_dict.items():
    print(f"{k}: {v}")
    print(f"{k}: {len(v)} locations")

Wheat: ['5742 Kölliken', '5074 Eiken', '5610 Wohlen', '4800 Zofingen', '3054 Schüpfen', '3076 Worb', '3114 Wichtrach', '3421 Lyssach', '4538 Oberbipp', '4917 Melchnau']
Wheat: 10 locations
Barley: ['5742 Kölliken', '5074 Eiken', '5610 Wohlen', '4800 Zofingen', '3054 Schüpfen', '3415 Hasle-Rüegsau', '3076 Worb', '3114 Wichtrach', '3421 Lyssach', '4538 Oberbipp']
Barley: 10 locations
Corn: ['5742 Kölliken', '5074 Eiken', '5610 Wohlen', '3076 Worb', '3315 Bätterkinden', '4538 Oberbipp', '4917 Melchnau', '4950 Huttwil', '3177 Laupen', '3423 Ersigen']
Corn: 10 locations
Soybean: ['5074 Eiken', '5610 Wohlen', '4800 Zofingen', '3415 Hasle-Rüegsau', '3076 Worb', '4538 Oberbipp', '4917 Melchnau', '4950 Huttwil', '3292 Busswil b. Büren', '3661 Uetendorf']
Soybean: 10 locations
Rapeseed: ['3076 Worb', '4538 Oberbipp', '4917 Melchnau', '9494 Schaan (FL)', '1784 Courtepin', '2900 Porrentruy', '8560 Märstetten', '1350 Orbe', '1580 Avenches', '8155 Niederhasli']
Rapeseed: 10 locations
Sunflowerseed: 

In [325]:
selected_crop = ""

def confirm_selection(b):
    global selected_crop
    selected_crop = dropdown.value

dropdown = widgets.Dropdown(
    options=['Wheat', 'Barley', 'Corn', 'Soybean', 'Rapeseed', 'Sunflowerseed'],
    description='Select the crop and check its risk index in {} days:'.format(N)
)

button = widgets.Button(description="Confirm Selection")
button.on_click(confirm_selection)

display(dropdown, button)

Dropdown(description='Select the crop and check its risk index in 14 days:', options=('Wheat', 'Barley', 'Corn…

Button(description='Confirm Selection', style=ButtonStyle())

In [328]:
print("Risk in index for", selected_crop, "in {} days".format(N))

Risk in index for Wheat in 14 days


In [329]:
geolocator = Nominatim(user_agent="geoapi", timeout=10)

cache = {}

def clean_location(address):
    address = address.replace("b.", "bei")
    if "(FL)" in address:
        address = address.replace("(FL)", "").strip()
        country = "Liechtenstein"
    else:
        country = "Switzerland"
    return address, country

def get_lat_lon_cached(address):
    if address in cache:
        return cache[address]

    try:
        clean_addr, country = clean_location(address)
        location = geolocator.geocode(f"{clean_addr}, {country}")
        if location:
            result = (location.latitude, location.longitude)
        else:
            result = (None, None)
    except:
        result = (None, None)

    manual_fix = {
        "3292 Busswil bei Büren": (47.133, 7.246),
        "9494 Schaan": (47.166, 9.516)
    }
    if result == (None, None) and clean_addr in manual_fix:
        result = manual_fix[clean_addr]

    cache[address] = result
    return result

print(f"Processing {selected_crop}...")

coords_list = []
for loc in tqdm(locations_dict[selected_crop]):
    lat, lon = get_lat_lon_cached(loc)
    coords_list.append([loc, lat, lon])

selected_crop_loc = pd.DataFrame(coords_list, columns=["Standort", "Latitude", "Longitude"])

print(" ")
print("The locations of farms in Switzerland for {}".format(selected_crop))
print(" ")
print(selected_crop_loc)

Processing Wheat...


100%|██████████| 10/10 [00:09<00:00,  1.03it/s]

 
The locations of farms in Switzerland for Wheat
 
         Standort   Latitude  Longitude
0   5742 Kölliken  47.334149   8.021650
1      5074 Eiken  47.534331   7.987837
2     5610 Wohlen  47.350194   8.279049
3   4800 Zofingen  47.286283   7.948603
4   3054 Schüpfen  47.039854   7.375502
5       3076 Worb  46.930236   7.566273
6  3114 Wichtrach  46.844390   7.574317
7    3421 Lyssach  47.066115   7.581815
8   4538 Oberbipp  47.259886   7.662658
9   4917 Melchnau  47.181497   7.848776


***Input 2: Climate Data***

* Historical climate data

In [330]:
weather_rows = []
columns_keep = ["tavg", "tmin", "tmax", "prcp", "snow", "wspd", "wpgt"]

for idx, row in selected_crop_loc.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    start = datetime(year, 1, 1)
    end = datetime(year, 12, 31)
    stations = Stations().nearby(lat, lon).fetch(10)
    if stations.empty:
        print(f"No stations found for {lat},{lon}")
        continue
    station_means = []
    for sid in stations.index:
        try:
            data = Daily(sid, start, end).fetch()
            if data.empty:
                continue
            yearly_mean = data[columns_keep].mean()
            station_means.append(yearly_mean)
        except Exception as e:
            print(f"Failed for station {sid}: {e}")
    if not station_means:
        continue
    combined_mean = pd.concat(station_means, axis=1).mean(axis=1, skipna=True)
    weather_data = {
        "Standort": row["Standort"],
        "Latitude": lat,
        "Longitude": lon
    }
    weather_data.update(combined_mean.to_dict())
    weather_rows.append(weather_data)
    if len(weather_rows) >= loc_num:
        break

weather_df = pd.DataFrame(weather_rows)
if "snow" in weather_df.columns:
    weather_df["snow"].fillna(0.025, inplace=True)

print(weather_df)

         Standort   Latitude  Longitude       tavg      tmin       tmax  \
0   5742 Kölliken  47.334149   8.021650  11.393887  7.236445  15.984835   
1      5074 Eiken  47.534331   7.987837  11.448713  7.177498  16.136100   
2     5610 Wohlen  47.350194   8.279049  11.155713  7.050572  15.631616   
3   4800 Zofingen  47.286283   7.948603  10.993097  6.940142  15.409472   
4   3054 Schüpfen  47.039854   7.375502  10.571420  6.475314  15.025539   
5       3076 Worb  46.930236   7.566273  10.176488  6.134242  14.610463   
6  3114 Wichtrach  46.844390   7.574317  10.098732  6.162995  14.459955   
7    3421 Lyssach  47.066115   7.581815  10.346545  6.156621  14.961115   
8   4538 Oberbipp  47.259886   7.662658  11.332749  6.945504  16.071547   
9   4917 Melchnau  47.181497   7.848776  10.403139  6.388470  14.804644   

       prcp       snow      wspd       wpgt  
0  2.634247   0.025000  6.782579  25.277526  
1  2.909954  20.176152  7.222095  25.520316  
2  2.360261   0.197802  7.187400  26

The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




***Collection of extrem weather days***

In [331]:
"""
OPEN_METEO_ARCHIVE = "https://archive-api.open-meteo.com/v1/archive"

THRESH = {
    "heavy_rain_mm": 50.0,
    "heavy_snow_cm": 20.0,
    "heatwave_temp_c": 35.0,
    "strong_wind_ms": 15.0,
    "extreme_uv_index": 8.0,
    "dry_day_precip_mm": 1.0,
    "dry_fraction_for_drought": 0.8,
}

def get_yearly_weather_and_extremes(lat, lon, year, thresholds=None, timezone="Europe/Zurich"):
    if thresholds is None:
        thresholds = THRESH
    else:
        tmp = THRESH.copy()
        tmp.update(thresholds)
        thresholds = tmp

    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": ",".join([
            "temperature_2m_max","temperature_2m_min",
            "precipitation_sum","snowfall_sum",
            "wind_speed_10m_max","uv_index_max"
        ]),
        "timezone": timezone
    }

    r = requests.get(OPEN_METEO_ARCHIVE, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    daily = data.get("daily", {})
    if not daily:
        raise ValueError("Open-Meteo returned no daily data for the given coordinates/year.")

    df = pd.DataFrame(daily)

    mean_data = {
        "mean_temperature_2m_max": df["temperature_2m_max"].mean(),
        "mean_temperature_2m_min": df["temperature_2m_min"].mean(),
        "mean_precipitation_sum": df["precipitation_sum"].mean(),
        "mean_snowfall_sum": df["snowfall_sum"].mean(),
        "mean_wind_speed_10m_max": df["wind_speed_10m_max"].mean(),
        "mean_uv_index_max": df["uv_index_max"].mean()
    }

    df["extreme_uv"] = df["uv_index_max"] >= thresholds["extreme_uv_index"]
    df["heavy_rain"] = df["precipitation_sum"] >= thresholds["heavy_rain_mm"]
    df["heavy_snow"] = df["snowfall_sum"]/10 >= thresholds["heavy_snow_cm"]  # 转为cm
    df["extreme_heat"] = df["temperature_2m_max"] >= thresholds["heatwave_temp_c"]
    df["strong_wind"] = df["wind_speed_10m_max"] >= thresholds["strong_wind_ms"]
    df["dry_day"] = df["precipitation_sum"] < thresholds["dry_day_precip_mm"]

    total_days = len(df)
    dry_days = int(df["dry_day"].sum())
    dry_fraction = dry_days / total_days if total_days > 0 else None
    simple_drought_flag = dry_fraction is not None and dry_fraction >= thresholds["dry_fraction_for_drought"]

    extreme_summary = {
        "period_days": total_days,
        "heavy_rain_days": int(df["heavy_rain"].sum()),
        "heavy_snow_days": int(df["heavy_snow"].sum()),
        "extreme_uv_days": int(df["extreme_uv"].sum()),
        "extreme_heat_days": int(df["extreme_heat"].sum()),
        "strong_wind_days": int(df["strong_wind"].sum()),
        "dry_days": dry_days,
        "dry_fraction": dry_fraction,
        "simple_drought_detected": bool(simple_drought_flag)
    }

    result = {**mean_data, **extreme_summary}
    return result

summary_rows = []
for idx, row in selected_crop_loc.iterrows():
    try:
        weather_summary = get_yearly_weather_and_extremes(
            lat=row["Latitude"], lon=row["Longitude"], year=year
        )
        row_data = {
            "Standort": row["Standort"],
            "Latitude": row["Latitude"],
            "Longitude": row["Longitude"]
        }
        row_data.update(weather_summary)
        summary_rows.append(row_data)
        time.sleep(1)
    except requests.exceptions.HTTPError as e:
        print(f"Request failed for {row['Standort']}: {e}")
        time.sleep(1)

extreme_weather_df = pd.DataFrame(summary_rows)
print(extreme_weather_df)
"""

'\nOPEN_METEO_ARCHIVE = "https://archive-api.open-meteo.com/v1/archive"\n\nTHRESH = {\n    "heavy_rain_mm": 50.0,\n    "heavy_snow_cm": 20.0,\n    "heatwave_temp_c": 35.0,\n    "strong_wind_ms": 15.0,\n    "extreme_uv_index": 8.0,\n    "dry_day_precip_mm": 1.0,\n    "dry_fraction_for_drought": 0.8,\n}\n\ndef get_yearly_weather_and_extremes(lat, lon, year, thresholds=None, timezone="Europe/Zurich"):\n    if thresholds is None:\n        thresholds = THRESH\n    else:\n        tmp = THRESH.copy()\n        tmp.update(thresholds)\n        thresholds = tmp\n\n    start_date = f"{year}-01-01"\n    end_date = f"{year}-12-31"\n\n    params = {\n        "latitude": lat,\n        "longitude": lon,\n        "start_date": start_date,\n        "end_date": end_date,\n        "daily": ",".join([\n            "temperature_2m_max","temperature_2m_min",\n            "precipitation_sum","snowfall_sum",\n            "wind_speed_10m_max","uv_index_max"\n        ]),\n        "timezone": timezone\n    }\n\n  

***Weather forecast in the future N days***

In [332]:
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"
OPEN_METEO_FORECAST = "https://api.open-meteo.com/v1/forecast"

def get_forecast_weather(latitude, longitude, n_days):
    start_date = date.today()
    end_date = start_date + timedelta(days=n_days - 1)

    daily_vars = [
        "temperature_2m_max","temperature_2m_min",
        "precipitation_sum","windspeed_10m_max",
        "weathercode","snowfall_sum"
    ]

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": ",".join(daily_vars),
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "timezone": "Europe/Zurich"
    }

    resp = requests.get(FORECAST_URL, params=params, timeout=30)
    resp.raise_for_status()

    df = pd.DataFrame(resp.json()["daily"])
    df["time"] = pd.to_datetime(df["time"]).dt.date
    def code_to_simple(wc, precip, wind, snow):
        if snow > 0:
            return "snow"
        elif precip > 5:
            return "rain"
        elif precip > 0 and wind > 10:
            return "showers/windy"
        elif wc in [0,1,2]:  # clear/sunny
            return "sunny"
        else:
            return "cloudy"

    df["weather_simple"] = df.apply(
        lambda row: code_to_simple(
            row["weathercode"],
            row["precipitation_sum"],
            row["windspeed_10m_max"],
            row.get("snowfall_sum", 0)
        ),
        axis=1
    )

    return df

In [333]:
THRESHOLDS = {
    "heatwave_temp_c": 35.0,
    "strong_wind_ms": 15.0,
    "heavy_rain_mm": 50.0,
    "heavy_snow_cm": 20.0
}

def extreme_weather_alert(lat, lon, n_days=N, timezone="Europe/Zurich"):
    start_date = date.today()
    end_date = start_date + timedelta(days=n_days-1)

    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,precipitation_sum,snowfall_sum,windspeed_10m_max",
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "timezone": timezone
    }

    resp = requests.get(OPEN_METEO_FORECAST, params=params)
    resp.raise_for_status()
    data = resp.json()

    df = pd.DataFrame(data["daily"])

    alerts = []
    for i, row in df.iterrows():
        msg = []
        if row["temperature_2m_max"] >= THRESHOLDS["heatwave_temp_c"]:
            msg.append("🔥 Heatwave")
        if row["windspeed_10m_max"] >= THRESHOLDS["strong_wind_ms"]:
            msg.append("💨 Strong wind")
        if row["precipitation_sum"] >= THRESHOLDS["heavy_rain_mm"]:
            msg.append("🌧️ Heavy rain")
        if (row["snowfall_sum"] / 10.0) >= THRESHOLDS["heavy_snow_cm"]:  # mm→cm
            msg.append("❄️ Heavy snow")

        alerts.append(", ".join(msg) if msg else "😃 NA")

    df["extreme_alert"] = alerts
    return df

In [334]:
forecast_rows = []
for idx, row in selected_crop_loc.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    standort = row["Standort"]
    try:
        df_forecast = get_forecast_weather(lat, lon, N)
        df_alert = extreme_weather_alert(lat, lon, N)
        df_merged = pd.merge(df_forecast, df_alert[["time", "extreme_alert"]], on="time", how="left")
        df_merged["Standort"] = standort
        df_merged["Latitude"] = lat
        df_merged["Longitude"] = lon
        forecast_rows.append(df_merged)
        time.sleep(1)
    except Exception as e:
        print(f"Forecast failed for {standort}: {e}")
        time.sleep(1)

forecast_all = pd.concat(forecast_rows, ignore_index=True)
forecast_all = forecast_all.drop(columns=["Latitude", "Longitude", "weathercode"])
cols = ["Standort"] + [c for c in forecast_all.columns if c != "Standort"]
forecast_all = forecast_all[cols]

          Standort        time  temperature_2m_max  temperature_2m_min  \
0    5742 Kölliken  2025-10-03                11.9                 1.9   
1    5742 Kölliken  2025-10-04                15.9                 8.9   
2    5742 Kölliken  2025-10-05                11.2                 6.9   
3    5742 Kölliken  2025-10-06                13.1                 8.1   
4    5742 Kölliken  2025-10-07                17.3                 6.9   
..             ...         ...                 ...                 ...   
135  4917 Melchnau  2025-10-12                20.1                10.8   
136  4917 Melchnau  2025-10-13                18.6                 9.8   
137  4917 Melchnau  2025-10-14                12.3                 7.7   
138  4917 Melchnau  2025-10-15                 7.7                 6.8   
139  4917 Melchnau  2025-10-16                 8.3                 6.5   

     precipitation_sum  windspeed_10m_max  snowfall_sum weather_simple  \
0                  0.0               

***Input 3: Yield Data (Optional but Recommended)***

In [340]:
df = pd.read_csv("FAOSTAT_data.csv")

mapping = {
    "Wheat": "Wheat",
    "Barley": "Barley",
    "Maize (corn)": "Corn",
    "Green corn (maize)": "Corn",
    "Corn": "Corn",
    "Soya beans": "Soya",
    "Soya bean oil": "Soya",
    "Rapeseed or canola oil, crude": "Rapeseed",
    "Sunflower seed": "Sunflower",
    "Sunflower-seed oil, crude": "Sunflower",
    "Beer of barley, malted": "Barley"
}


keywords = ["Wheat", "Barley", "Corn", "Soya", "Rapeseed", "Sunflower"]
mask = df["Item"].str.contains('|'.join(keywords), case=False, na=False)
df_filtered = df[mask]

columns_to_keep = ["Area", "Item", "Year", "Unit", "Value"]
df_filtered = df_filtered[columns_to_keep]

df_filtered_t = df_filtered[df_filtered["Unit"] == "t"].copy()

df_filtered_t["Item_clean"] = df_filtered_t["Item"].map(mapping)

df_final = df_filtered_t.groupby(["Year", "Item_clean"], as_index=False)["Value"].sum()
value_row = df_final[(df_final["Year"] == year) & (df_final["Item_clean"] == selected_crop)]
value = value_row["Value"].iloc[0]
print(value)


486667.0


***Input 4: Administrative or grid boundaries (crop location)***


In [336]:
ee.Authenticate()
ee.Initialize(project='trusty-sentinel-473919-a6')

def compute_ndvi_df(selected_crop_loc, selected_crop, year, total_yield=value, buffer_m=500):
    """
    selected_crop_loc: pd.DataFrame with columns Standort, Latitude, Longitude
    selected_crop: str, e.g., "Corn"
    year: int
    total_yield: float, total yield for this crop
    buffer_m: int, buffer in meters for NDVI averaging
    Returns: pd.DataFrame with Standort, crop, ndvi, estimated_yield
    """
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    all_points = ee.Geometry.MultiPoint([[row["Longitude"], row["Latitude"]] for idx, row in selected_crop_loc.iterrows()])

    collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterDate(start_date, end_date)
                  .filterBounds(all_points)
                  .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')))

    results = []
    for idx, row in selected_crop_loc.iterrows():
        point = ee.Geometry.Point([row["Longitude"], row["Latitude"]])
        ndvi_ts = collection.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point.buffer(buffer_m),
            scale=10
        )
        ndvi_value = ndvi_ts.get("NDVI").getInfo()
        results.append({
            "Standort": row["Standort"],
            "crop": selected_crop,
            "ndvi": ndvi_value
        })

    valid_points = [r for r in results if r["ndvi"] is not None]
    if valid_points:
        ndvi_sum = sum(r["ndvi"] for r in valid_points)
        for r in valid_points:
            r["estimated_yield"] = (r["ndvi"] / ndvi_sum) * total_yield
    else:
        print("⚠️ All NDVI values are None!")

    return pd.DataFrame(results)

In [341]:
df_ndvi = compute_ndvi_df(selected_crop_loc, selected_crop, year=year, total_yield=value)
print(df_ndvi)

         Standort   crop      ndvi  estimated_yield
0   5742 Kölliken  Wheat  0.298157     47586.062955
1      5074 Eiken  Wheat  0.306039     48843.926460
2     5610 Wohlen  Wheat  0.237234     37862.723316
3   4800 Zofingen  Wheat  0.268091     42787.447475
4   3054 Schüpfen  Wheat  0.338790     54071.141344
5       3076 Worb  Wheat  0.305803     48806.397113
6  3114 Wichtrach  Wheat  0.326623     52129.164765
7    3421 Lyssach  Wheat  0.344075     54914.627045
8   4538 Oberbipp  Wheat  0.284338     45380.573692
9   4917 Melchnau  Wheat  0.340130     54284.935835
